In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

import networkx as nx  # Library for hanling graphs (please check 4_Introduction_Graph)
import random # possibility to create random numbers (The "P" in PRM)
import numpy as np
import math

# sort nearest neighbour brute force
import heapq

# reduce coding effort by using function provided by scipy
from scipy.spatial.distance import euclidean

In [ ]:
def inSameConnectedComponent(node1, node2, graph):
    """ Check whether to nodes are part of the same connected component using
        functionality from NetworkX
    """
    for connectedComponent in nx.connected_components(graph):
        if (node1 in connectedComponent) & (node2 in connectedComponent):
            return True
        
    return False

In [ ]:
import random

def Bridge_sampling(collChecker):
    
    limits = collChecker.getEnvironmentLimits()        
    pos = [random.uniform(limit[0],limit[1]) for limit in limits]
    
    
    #get a colliding point
    for t in range(0,200):
        while not collChecker.pointInCollision(pos):
            pos = [random.uniform(limit[0],limit[1]) for limit in limits]
        
        #store the x and y value of the colliding point
        pos_x=pos[0]
        pos_y=pos[1]
    
        #find a second coliding Point in the neigborhood
        for i in range(0,200):
        #get a distance over a gaussian distribution   
            me,sigma = 0,1 #Mean value of the gaussian distribution, standard deviation 
            d=np.random.normal(me,sigma) #get a random distance form a gaussian distribution 
            for n in range(0,12):
                #alpha=random.uniform(0,360)*(180/math.pi) #get an random angle in rad
                alpha=n*90*(180/math.pi)#get an angle in rad 
                pos2_x=d*math.cos(alpha)+pos_x
                pos2_y=d*math.sin(alpha)+pos_y
                pos2=[pos2_x,pos2_y]
                if  collChecker.pointInCollision(pos2): #return point when in collison 
                    break
            pos3_x=(pos2_x+pos_x)/2
            pos3_y=(pos2_y+pos_y)/2   
            pos3=[pos3_x,pos3_y]
            if  collChecker.pointInCollision(pos3): #return point when in collison 
                    return pos3
            

        
    
    
    #if no point found with gaussian pick a random collison free point
    while not collChecker.pointInCollision(pos):
        pos = [random.uniform(limit[0],limit[1]) for limit in limits]
    return pos
            
            
   